# 0.0 Imports

## 0.2 Load Dataset

In [2]:
import re
import pickle
import sqlite3

import pandas            as pd
import numpy             as np
import seaborn           as sns
import umap.umap_        as umap

from plotly              import express       as px
from sklearn             import cluster       as c
from sklearn             import metrics       as m
from sklearn             import preprocessing as pp
from sklearn             import decomposition as dd
from sklearn             import ensemble      as en
from sklearn             import mixture       as mx
from matplotlib          import pyplot        as plt

from sqlalchemy          import create_engine
from sklearn.manifold    import TSNE

class insiders( object ):
    def __init__( self ):
        self.home_path            = 'C:/Users/t.mesha09/Downloads/Estudos DS/Programa de Fidelidade/'
        self.gross_revenue_scaler = pickle.load(open(self.home_path+'data/models/gross_revenue_scaler.pkl', 'rb'))
        self.recency_days_scaler  = pickle.load(open(self.home_path+'data/models/recency_days_scaler.pkl', 'rb'))
        self.qtde_produtos_scaler = pickle.load(open(self.home_path+'data/models/qtde_produtos_scaler.pkl', 'rb'))
        self.frequency_scaler     = pickle.load(open(self.home_path+'data/models/frequency_scaler.pkl', 'rb'))
        self.qtde_returns_scaler  = pickle.load(open(self.home_path+'data/models/qtde_returns.pkl', 'rb'))
        self.rf_model             = pickle.load(open(self.home_path+'data/models/rf_model.pkl', 'rb'))
        self.reducer              = pickle.load(open(self.home_path+'data/models/umap_reducer.pkl', 'rb'))
    
    
    def data_cleaning( self, df1 ): 
        # rename columns
        cols_new = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date', 'unit_price', 'customer_id', 'country']
        df1.columns = cols_new
        
        # replace NA
        df_missing = df1.loc[df1['customer_id'].isna(),:]
        df_not_missing = df1.loc[~df1['customer_id'].isna(),:]
        
        # create reference - Adicionando um valor para customer_id que não tenha na base. Como o máx é 19000, o primeiro id missing que aparecer será o 19001 e assim por diante.
        df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
        df_backup['customer_id'] = np.arange(19000, 19000+len(df_backup),1)

        # merge original with reference dataframe
        df1 = pd.merge(df1, df_backup, on='invoice_no', how='left')

        # coalesce
        df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

        # drop extra columns
        df1 = df1.drop(columns=['customer_id_x', 'customer_id_y'])
        
        # change dtypes
        # invoice_date
        df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')

        # customer_id
        df1['customer_id'] = df1['customer_id'].astype('int64')
        return df1 
        
    def data_filtering( self, df2 ):
        # unit price > 0.01
        df2 = df2.loc[df2['unit_price'] >= 0.04, :]

        # stock code != [POST, D, M, DOT, CRUK]
        df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
               'DCGSSGIRL', 'PADS', 'B', 'CRUK'])]

        # description
        df2 = df2.drop(columns='description', axis=1)

        # map
        df2 = df2[~df2['country'].isin(['European Community', 'Unspecified'])]

        # bad users - Pessoas com comportamentos estranho na analise exploratória de dados
        df2 = df2[~df2['customer_id'].isin([16446 ])]

        ## quantity - Negative number means product returns
        df2_returns = df2.loc[df2['quantity'] < 0, :]
        df2_purchase = df2.loc[df2['quantity'] >= 0, :]
        
        return df2
        
    def feature_engineering(self, df3):
        #Será criado uma tabela de referência, com a menor granularidade possível
        df_ref = df3.drop(['invoice_no', 'stock_code', 
                   'quantity', 'invoice_date', 'unit_price', 
                   'country'], axis = 1).drop_duplicates(ignore_index=True)
        
        # Gross Revenue (Faturamento) = quantity * price
        df2_purchase.loc[:, 'gross_revenue'] = df2_purchase.loc[:,'quantity']*df2_purchase.loc[:,'unit_price']

        # Monetary (Quanto esta pessoa gastou na loja)
        df_monetary = df2_purchase.loc[:,['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()

        # Adicionando no datafram de referencia
        df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')
       
        # Recency - Last Day Purchase
        # Como o dataset é antigo, será escolhido a data como a última data de compra do dataset. Esta data servirá de referencia, uma vez que o recorte dos dados é de dois anos atrás.
        # em um projeto real, é selecionado o datetime.today(), selecionando assim a data de HOJE.
        df_recency = df2_purchase[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
        df_recency['recency_days'] = (df2_purchase['invoice_date'].max()-df_recency['invoice_date']).dt.days
        df_recency = df_recency[['customer_id', 'recency_days']].copy()
        df_ref = pd.merge(df_ref, df_recency, how='left', on='customer_id')
        
        # Numero de produtos
        df_frequency = df2_purchase[['customer_id', 'stock_code']].groupby('customer_id').count().reset_index().rename(columns={'stock_code': 'qtde_produtos'})
        df_ref = pd.merge(df_ref, df_frequency, how='left', on='customer_id')
        df_ref.isna().sum()
        
        df_aux = (df2_purchase[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                            .groupby('customer_id')
                                                            .agg(max_  = ('invoice_date', 'max'), 
                                                                 min_  = ('invoice_date', 'min'), 
                                                                 days_ = ('invoice_date', lambda x: (x.max() - x.min()).days +1),
                                                                 buy_  = ('invoice_no', 'count') )  ).reset_index()

        # frequency
        df_aux['frequency'] = df_aux[['buy_', 'days_']].apply(lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis=1)

        # merge
        df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left') 
        
        # number of returns
        df_returns = df2_returns[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity': 'qtde_returns'})
        df_returns['qtde_returns'] = df_returns['qtde_returns']*(-1)

        df_ref = pd.merge(df_ref, df_returns, how='left', on='customer_id')
        df_ref.loc[df_ref['qtde_returns'].isna(),'qtde_returns'] = 0
        
        return df_ref
    
    def eda( self, df4 ):
        cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'qtde_produtos', 'frequency', 'qtde_returns']
        df43 = df4[cols_selected].drop(columns='customer_id', axis=1)
        
        # espaço de features
        mm = pp.MinMaxScaler()

        df43['gross_revenue'] = self.gross_revenue_scaler.transform(df43[['gross_revenue']])

        df43['recency_days']  = self.recency_days_scaler.transform(df43[['recency_days']])

        df43['qtde_produtos'] = self.qtde_produtos_scaler.transform(df43[['qtde_produtos']])

        df43['frequency']     = self.frequency_scaler.transform(df43[['frequency']])

        df43['qtde_returns']  = self.qtde_returns_scaler.transform(df43[['qtde_returns']])
        
        # training dataset
        X = df43.drop(columns = ['gross_revenue'], axis=1)
        y = df43['gross_revenue']

        # leaf
        df_leaf = pd.DataFrame(self.rf_model.apply(X))
        
        # reducer dimensionality
        embedding = self.reducer.transform(df_leaf)

        # embedding
        df_tree = pd.DataFrame()
        df_tree['embedding_x'] = embedding[:,0]
        df_tree['embedding_y'] = embedding[:,1]
        
        df5 = df_tree.copy()
        
        return df5
    
    def model_training(self, df8):
        X = df_tree.copy()
        
        k = 8
        # model definition
        gmm_model = mx.GaussianMixture(n_components=k, n_init = 300, random_state=32)

        # model training
        gmm_model.fit(X)

        # model predict
        labels = gmm_model.predict(X)
        
        # cluster analysis
        df9 = X.copy()
        df9['cluster'] = labels
        
        df92 = df4[cols_selected].copy()
        df92['cluster'] = labels

        # change dtypes
        df92['recency_days'] = df92['recency_days'].astype(int)
        df92['qtde_produtos'] = df92['qtde_produtos'].astype(int)
        df92['qtde_returns'] = df92['qtde_returns'].astype(int)
        
        return df92

C:\Users\t.mesha09\Anaconda3\envs\pa005-programa_de_fidelidade\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# read data
df_raw = pd.read_csv('../data/teste-Ecommerce.csv', encoding='unicode_escape', sep=';')

# drop exatra column
#df_raw = df_raw.drop('Unnamed: 8', axis=1)

In [11]:
df_raw

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,905263,85123A,WHITE HANGING HEART T-LIGHT HOLDER,4,30/nov/16,2.55,17850,United Kingdom
1,905264,71053,WHITE METAL LANTERN,7,29/dez/16,3.39,17850,United Kingdom
2,905265,84406B,CREAM CUPID HEARTS COAT HANGER,5,05/dez/16,2.75,17850,United Kingdom
3,905266,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,06/dez/16,3.39,17850,United Kingdom
4,905267,84029E,RED WOOLLY HOTTIE WHITE HEART.,3,07/nov/16,3.39,17850,United Kingdom
5,905268,22752,SET 7 BABUSHKA NESTING BOXES,4,08/nov/16,7.65,17850,United Kingdom
6,905269,21730,GLASS STAR FROSTED T-LIGHT HOLDER,5,07/dez/16,4.25,17850,United Kingdom
7,905270,22633,HAND WARMER UNION JACK,4,07/dez/16,1.85,17850,United Kingdom
8,905271,22632,HAND WARMER RED POLKA DOT,5,29/nov/16,1.85,17850,United Kingdom
9,905272,84879,ASSORTED COLOUR BIRD ORNAMENT,66,29/nov/16,1.69,13047,United Kingdom


In [12]:
insiders( df_raw )

TypeError: __init__() takes 1 positional argument but 2 were given